# --- Day 21: Scrambled Letters and Hash ---

The computer system you're breaking into uses a weird scrambling function to store its passwords. It shouldn't be much trouble to create your own scrambled password so you can add it to the system; you just have to implement the scrambler.

The scrambling function is a series of operations (the exact list is provided in your puzzle input). Starting with the password to be scrambled, apply each operation in succession to the string. The individual operations behave as follows:

- swap position X with position Y means that the letters at indexes X and Y (counting from 0) should be swapped.
- swap letter X with letter Y means that the letters X and Y should be swapped (regardless of where they appear in the string).
- rotate left/right X steps means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
- rotate based on position of letter X means that the whole string should be rotated to the right based on the index of letter X (counting from 0) as determined before this instruction does any rotations. Once the index is determined, rotate the string to the right one time, plus a number of times equal to that index, plus one additional time if the index was at least 4.
- reverse positions X through Y means that the span of letters at indexes X through Y (including the letters at X and Y) should be reversed in order.
- move position X to position Y means that the letter which is at index X should be removed from the string, then inserted such that it ends up at index Y.

For example, suppose you start with abcde and perform the following operations:

- swap position 4 with position 0 swaps the first and last letters, producing the input for the next step, ebcda.
- swap letter d with letter b swaps the positions of d and b: edcba.
- reverse positions 0 through 4 causes the entire string to be reversed, producing abcde.
- rotate left 1 step shifts all letters left one position, causing the first letter to wrap to the end of the string: bcdea.
- move position 1 to position 4 removes the letter at position 1 (c), then inserts it at position 4 (the end of the string): bdeac.
- move position 3 to position 0 removes the letter at position 3 (a), then inserts it at position 0 (the front of the string): abdec.
- rotate based on position of letter b finds the index of letter b (1), then rotates the string right once plus a number of times equal to that index (2): ecabd.
- rotate based on position of letter d finds the index of letter d (4), then rotates the string right once, plus a number of times equal to that index, plus an additional time because the index was at least 4, for a total of 6 right rotations: decab.

After these steps, the resulting scrambled password is decab.

Now, you just need to generate a new scrambled password and you can access the system. Given the list of scrambling operations in your puzzle input, **what is the result of scrambling abcdefgh?**

In [63]:
with open(f'inputs/21.txt') as f:
    data = f.read().strip().split("\n")
data[:18]

['rotate based on position of letter a',
 'swap letter g with letter d',
 'move position 1 to position 5',
 'reverse positions 6 through 7',
 'move position 5 to position 4',
 'rotate based on position of letter b',
 'reverse positions 6 through 7',
 'swap letter h with letter f',
 'swap letter e with letter c',
 'reverse positions 0 through 7',
 'swap position 6 with position 4',
 'rotate based on position of letter e',
 'move position 2 to position 7',
 'swap position 6 with position 4',
 'rotate based on position of letter e',
 'reverse positions 2 through 3',
 'rotate right 2 steps',
 'swap position 7 with position 1']

In [112]:
test_ins = """
swap position 4 with position 0
swap letter d with letter b
reverse positions 0 through 4
rotate left 1 step
move position 1 to position 4
move position 3 to position 0
rotate based on position of letter b
rotate based on position of letter d
"""
test_ins = test_ins.strip().splitlines()

first up, a couple of helper instructions:

In [31]:
import numpy as np
import regex

In [48]:
def get_nums(line):
    """takes in a string and returns 2 numbers, defaults to 0 if num not found """
    return [int(i) for i in (line + " 0 0").split(" ") if i.isdigit()][:2]
    
get_nums("this is a test 44 line with 55 numbers"), get_nums("no nums here")

([44, 55], [0, 0])

In [62]:
def swap(pw, x,y):
    "takes in a list pw and two index nums, swaps the letters at those index"
    
    # if x or y is not a position index, get the position
    if type(x) is str:
        x = pw.index(x)
    
    if type(y) is str:
        y = pw.index(y)
    
    pw[x], pw[y] = pw[y], pw[x]
    return pw
    
swap(list("abcdefgh"), "a" ,"c"), swap(list("abcdefgh"), 1 ,4) 

(['c', 'b', 'a', 'd', 'e', 'f', 'g', 'h'],
 ['a', 'e', 'c', 'd', 'b', 'f', 'g', 'h'])

In [168]:
def solve_one(password="abcde", instructions=test_ins, verbose=False):
    pw = list(password)
    
    for line in instructions:
        if verbose: print(line)
        ins = line.startswith
        words = line.split()
        x,y = get_nums(line)
        
        if ins("swap position"):
            # swap the letters at position X, Y
            pw = swap(pw, x, y)
        elif ins("swap letter"):
            # find letters X and Y, then swap
            x, y = pw.index(words[2]), pw.index(words[5])
            pw = swap(pw, x, y)
        elif ins("rotate left"):
            # rotate left X times
            pw = list(np.roll(pw, -x))
        elif ins("rotate right"):
            # rotate right X times
            pw = list(np.roll(pw, x))
        elif ins("rotate based on position of letter"):
            # get index of letter, rotate 1 + index + (if index >=4 than add 1)
            i = int(pw.index(words[-1]))
            if i >= 4: i += 1
            pw = list(np.roll(pw, i+1))
        elif ins("reverse positions"):
            # reverse the letters from index X to Y, including x and y
            pw = pw[:x] + pw[x:y+1][::-1] + pw[y+1:]
        elif ins("move position"):
            #remove letter from X and insert at Y
            letter = pw.pop(x)
            pw.insert(y, letter)
        else:
            print(line)
            print("Bad stuff happening... break")
            break
            
        if verbose: print("".join(pw))
    
    return "".join(pw)
            
solve_one(verbose=True)

swap position 4 with position 0
ebcda
swap letter d with letter b
edcba
reverse positions 0 through 4
abcde
rotate left 1 step
bcdea
move position 1 to position 4
bdeac
move position 3 to position 0
abdec
rotate based on position of letter b
ecabd
rotate based on position of letter d
decab


'decab'

In [169]:
%time solve_one("abcdefgh", data)

CPU times: user 5 ms, sys: 2 ms, total: 7 ms
Wall time: 5.35 ms


'aefgbcdh'

ans to part 1: `aefgbcdh`

# --- Part Two ---

You scrambled the password correctly, but you discover that you can't actually modify the password file on the system. You'll need to un-scramble one of the existing passwords by reversing the scrambling process.

What is the un-scrambled version of the scrambled password `fbgdceah`?

All the instructions besides the `rotate based on position of letter` are straightforward to reverse. so thinking of how to do that:

In [159]:
part_two = "fbgdceah"

def un_scramble(password="decab", instructions=test_ins, verbose=False):
    pw = list(password)
    
    for line in instructions[::-1]:
        if verbose: print(line)
        ins = line.startswith
        words = line.split()
        x,y = get_nums(line)
        
        if ins("swap position"):
            # swap the letters at position X, Y
            pw = swap(pw, x, y)
        elif ins("swap letter"):
            # find letters X and Y, then swap
            x, y = pw.index(words[2]), pw.index(words[5])
            pw = swap(pw, x, y)
        elif ins("rotate left"):
            # rotate left X times
            pw = list(np.roll(pw, x))
        elif ins("rotate right"):
            # rotate right X times
            pw = list(np.roll(pw, -x))
        elif ins("rotate based on position of letter"):
            # get index of letter, rotate 1 + index + (if index >=4 than add 1)
            # TODO
            
            i = int(pw.index(words[-1]))
            if i % 2 == 0:
                
            pw = list(np.roll(pw, -i-1))
        elif ins("reverse positions"):
            # reverse the letters from index X to Y, including x and y
            pw = pw[:x] + pw[x:y+1][::-1] + pw[y+1:]
        elif ins("move position"):
            #remove letter from X and insert at Y
            letter = pw.pop(y)
            pw.insert(x, letter)
        else:
            print(line)
            print("Bad stuff happening... break")
            break
            
        if verbose: print("".join(pw))
    
    return "".join(pw)
            
un_scramble(verbose=True)

rotate based on position of letter d
ecabd
rotate based on position of letter b
decab
move position 3 to position 0
ecadb
move position 1 to position 4
ebcad
rotate left 1 step
debca
reverse positions 0 through 4
acbed
swap letter d with letter b
acdeb
swap position 4 with position 0
bcdea


'bcdea'

While I'm figuring out a decent unscramble function, here goes a brute force version:

In [180]:
from itertools import permutations

def brute_force(scrambeled="decab", data=test_ins):
    for pw in ("".join(i) for i in permutations(scrambeled)):
        if solve_one(pw, data) == scrambeled:
            #print(pw, scrambeled)
            return pw, scrambeled

brute_force()

('deabc', 'decab')

I'm getting two solutions for decab.., but lets see on the other ones:

In [178]:
%time brute_force("aefgbcdh", data=data)

CPU times: user 3.3 s, sys: 81 ms, total: 3.38 s
Wall time: 3.38 s


('abcdefgh', 'aefgbcdh')

In [174]:
%time brute_force("fbgdceah", data=data)

fbgdceah egcdahbf
CPU times: user 40.7 s, sys: 849 ms, total: 41.6 s
Wall time: 41.7 s


('egcdahbf', 'fbgdceah')

ans to part 2: `egcdahbf`